## Preparando Entorno
Para iniciar nuestro notebook de Google Colab, es necesario correr los comandos necesarios para instalar pytorch y librerias auxiliares.

In [ ]:
!pip install torch torchvision
!pip install matplotlib

Si se requiere verificar que el notebook cuente con GPU asignada, se puede correr el siguiente comando:

In [ ]:
!nvidia-smi

# Regresión logística (Logistic regresion)
Importar las librerias en python

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import matplotlib
import time

### PASO 1
Cargar el dataset MNIST

In [ ]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

### PASO 2
Hacer que nuestros dataset sean optimizados para un DataLoader (empaquetados)

In [ ]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

### PASO 3
Crear la clase del modelo

In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = self.linear(x)
        return out

### PASO 4
Inicializar el modelo

In [ ]:
input_dim = 28*28
output_dim = 10

model = LogisticRegressionModel(input_dim, output_dim)

#######################
#    SOLO PARA GPU    #
#######################
if torch.cuda.is_available():
    model.cuda()

### PASO 5
Inicializar el tipo de función de "Loss" y Optimizador

In [ ]:
## LOSS
criterion = nn.MSELoss()

## Optimizador
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### PASO 6
Entrenar el modelo...

In [ ]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #######################
        #    SOLO PARA GPU    #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.view(-1, 28*28).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, 28*28))
            labels = Variable(labels)
        
        # Limpiar los parametros en los gradientes
        optimizer.zero_grad()
        
        # FORWARD / INFERENCIA
        outputs = model(images)
        
        # Calcular la "perdida" (loss)
        loss = criterion(outputs, labels)
        
        # Obtener los parametros de los gradientes
        loss.backward()
        
        # Actualizar los parametros
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calcular precisión         
            correct = 0
            total = 0
            # Iterar sobre el dataset de evaluación
            for images, labels in test_loader:
                #######################
                #    SOLO PARA GPU    #
                #######################
                images = Variable(images.view(-1, 28*28).cuda())
                
                # FORWARD / INFERENCIA
                outputs = model(images)
                
                # Obtener predicciones esperadas
                _, predicted = torch.max(outputs.data, 1)
                
                # Numero de salidas
                total += labels.size(0)
                
                #######################
                #    SOLO PARA GPU    #
                #######################
                # Total de predicciones correctas
                correct += (predicted.cpu() == labels.cpu()).sum()
            
            accuracy = 100 * correct / total
            
            # DEBUG
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))